# COM-415 Final Project by Nicola Figari and Dohun Jeong

In [23]:
# These should be already installed on your computer
import matplotlib.pyplot as plt
import numpy as np
import numpy.fft as fft
import scipy as scipy
import scipy.io.wavfile as wav
import scipy.signal as sig
import time
import timeit
#import neopixels
import warnings
warnings.filterwarnings('ignore')
from IPython.display import Audio
from IPython.display import clear_output

# These may need to be installed
import pylab as py
from pylab import imread,subplot,imshow,show 
from ipywidgets import widgets #pip install ipywidgets
import pygame # pip install pygame
from pywt import _multilevel # pip install pywt

In [2]:
# This function takes file name string and imports audio file in the same directory
# returns, sampling rate and the audio file.
def importAudio(filename):
    return wav.read(filename)
# This function takes in a 2-element array-like datatype as 'seconds'. The first element indicate the starting time
# and the second element indicate the end time of the original audio file that needs to be clipped.
def clipAudio(fs, audio, seconds):
    return audio[seconds[0]*fs:seconds[1]*fs]

def plotAudio(filename):
    [fs, x] = wav.read(filename)
    m = x.shape[0]
    t = np.linspace(0.0, m/fs, m)
    fig, axes = plt.subplots(1, 1)
    axes.plot(t, x,'r')
    plt.show()

# Filterbank algorithm

In [3]:
def filterbank(signal, bandlimits, Fs):

    transform = fft.fft(signal)
    n = len(transform)
    n_bands = len(bandlimits)
    bl = np.zeros((n_bands))
    br = np.zeros((n_bands))
    for i in range(n_bands-1):
        bl[i] = (np.floor(bandlimits[i]/Fs*n/2)+1)
        br[i] = (np.floor(bandlimits[i+1]/Fs*n/2))
    bl = bl.astype(int)
    br = br.astype(int)
    bl[n_bands-1] = np.floor(bandlimits[n_bands-1]/Fs*n/2)+1
    br[n_bands-1] = np.floor(n/2)
    #print(bl, br, bandlimits[n_bands-1]/Fs*n/2+1, n/2)

    output = np.zeros((n,n_bands))

    for i in range(n_bands):
        output[bl[i]:br[i],i] = transform[bl[i]:br[i]]
        output[n+1-br[i]:n+1-bl[i],i] = transform[n+1-br[i]:n+1-bl[i]]

    output[0,0]=0;
    return output

In [4]:
def hwindow(signal, winlength, bandlimits, Fs):

    n = len(signal)
    n_bands = len(bandlimits)
    han_len = (winlength*2*Fs)//1

    hann = np.zeros((n))

    for a in range(han_len):
        hann[a] = np.square((np.cos(a*np.pi/han_len/2)))

    wave = np.real(fft.ifft(signal, axis=0))
    wave = np.absolute(wave)
    wave = fft.fft(wave, axis=0)
    wave = np.multiply(wave, fft.fft(hann)[:, np.newaxis])
    wave = np.real(fft.ifft(wave, axis=0))
    return wave

In [5]:
    # Input: audio is a 2D np.array, with each column being a frequency band
def diffrect(audio):
        audio = np.diff(audio, axis=0)
        audio[audio < 0] = 0
        return audio

In [6]:
def combfilter(audio, accuracy, minBPM, maxBPM, numBands, maxfreq):
    length = audio.shape[0]
    maxEnergy = 0
    npulse = 3
    for bpm in range(minBPM, maxBPM) :#modify loop to modify accuracy
        # try comb with given bpm
        period = (np.floor(60/bpm*2*maxfreq)).astype(int)

        result = np.zeros(((length+period*(npulse)).astype(int),numBands))
        for i in range(npulse):
            result[i*period:i*period+length] += audio
        energy = np.sum(np.abs(result)**2)

        if (energy > maxEnergy):
            currBPM = bpm
            maxEnergy = energy
    return currBPM

# Energy algorithm

The algorithm divides the data into blocks of 1000 samples and compares the energy of a block $E_j$ with the energy of a window of blocks $E_{avg}$ (48 blocks) to which the block itself belongs. The energy of a block is used to detect a beat. If the energy $E_j$ of a block is above a certain threshold then the block is considered to be a beat.
$$E_j = \sum_{i=0}^{1000}f(i)$$

$$E_{avg} = \frac{1}{48}\sum_{j=1}^{48}E_j$$

The threshold can be defined in two different ways: 
1. We consider the avarage energy of a window of blocks as the threshold 
$$E_j > E_{avg}$$
2. We take the avarage energy of a windows and weight it by a factor $c$ 
$$E_j > cE_{avg}$$

The factor $c$ is defined as follows: 
$$c=−0.0025714var(E)+1.5142857$$

$$var(E) = \frac{1}{48}\sum_{j=0}^{48}(E_{avg} - E_j)^2$$

$c$ depends on the energy variance of the window of blocks and quanties how marked the beats of the song are. The bigger the variance, the smaller the weight. 

Once we detected the blocks which correspond to beats and the ones which don't, we assign 1 to the beat blocks and 0 to the non beat blocks. We then pass the resulting signal through a comb filter that gives us the BPM.

In [7]:
def energy_beat(audio, fs):
    signal = audio.astype(float)
    n = np.floor(len(signal)/48000)
    signal = signal[0:(np.int(n)*48000)]
    signal = np.power(signal,2)
    inst = np.reshape(signal, (np.int(np.ceil(len(signal)/1000)),1000))
    Ej = np.sum(inst,1)
    a = np.reshape(Ej,(np.int((len(Ej)/48)),48))
    avgE = np.mean(a,1)
    avgE = np.tile(avgE,(48,1))
    avgE = np.reshape(avgE,len(Ej),'F')
    beat = np.greater(Ej, avgE)
    beat = beat.astype(int)
    beats = np.tile(beat,(1000,1))
    beats = np.reshape(beats,len(beat)*1000,'F')
    return beats

In [8]:
def energy_beat_w_variance(audio, fs):
    signal = audio.astype(float)
    n = np.floor(len(signal)/48000)
    signal = signal[0:(np.int(n)*48000)]
    signal = np.power(signal,2)
    inst = np.reshape(signal, (np.int(np.ceil(len(signal)/1000)),1000))
    Ej = np.sum(inst,1)
    a = np.reshape(Ej,(np.int((len(Ej)/48)),48))
    avgE = np.mean(a,1)
    avgE = np.tile(avgE,(48,1))
    avgE = np.reshape(avgE,len(Ej),'F')
    b = (avgE - Ej)**2
    d = np.reshape(b,(len(b)//48,48))

    var = np.var(inst,1)

    c = (-0.0025714)*var + 1.5142857
    beat = np.greater(Ej, c*avgE)
    beat = beat.astype(int)
    beats = np.tile(beat,(1000,1))
    beats = np.reshape(beats,len(beat)*1000,'F')

    return beats

In [9]:
def combfilterEnergy(audio, accuracy, minBPM, maxBPM, maxfreq):
    length = audio.shape[0]
    maxEnergy = 0
    npulse = 3
    for bpm in range(minBPM, maxBPM) :#modify loop to modify accuracy
        # try comb with given bpm
        period = (np.floor(60/bpm*2*maxfreq)).astype(int)

        result = np.zeros((length+period*(npulse)).astype(int))
        for i in range(npulse):
            result[i*period:i*period+length] += audio
        energy = np.sum(np.abs(result)**2)

        if (energy > maxEnergy):
            currBPM = bpm
            maxEnergy = energy
    return currBPM

# Discrete Wavelet Transform Method - Not fully implemented


In [10]:
def DWT(fs, audio):
    cA5, cD5, cD4, cD3, cD2, cD1, cD0 = _multilevel.wavedec(audio, 'db1', level=6)
    transform = np.zeros((len(audio)//2,6))
    transform[0:len(cD0),0] = cD0
    transform[0:len(cD1),1] = cD1
    transform[0:len(cD2),2] = cD2
    transform[0:len(cD3),3] = cD3
    transform[0:len(cD4),4] = cD4
    transform[0:len(cD5),5] = cD5
    return transform

In [11]:
def LPF(audio):
    return sig.lfilter([0.01],[1.99],audio,axis=0)

In [12]:
def FWR(audio):
    return np.absolute(audio)

In [13]:
def downsample(audio):
    for i in range(6):
        np.delete(audio[:,i], np.arange(0, len(audio[:,i]), 2**(6-i)))
    return audio

In [14]:
def normalize(audio):
    for i in range(6):
        audio[:,i] -= np.average(audio[:,i])
    return audio

In [15]:
def autocorrelate(fs, audio):
    output = np.zeros(len(audio))
    audio = np.sum(audio, axis=-1)
    #for i in range(len(audio)):
     #   delayedAudio = np.append(np.zeros(i), audio)
      #  audioPad = np.append(audio, np.zeros(i))
       # output[i] = np.sum(audioPad*delayedAudio)
    return np.abs(audio)

# Change of Spectrum Algorithm

In [16]:
def spectrumMethod(fs, audio):
    audioLength = len(audio)
    f, t, Zxx = sig.stft(audio, fs)
    Zxx =  np.log(1+1000*np.abs(Zxx) )
    audio = np.diff(Zxx, axis=1) #differentiate
    audio = np.sum(audio, axis=0) #accumulate
    audio -= np.average(audio) #normalize
    audio = sig.resample(audio, audioLength) #back to the same time rate
    bpm = combfilterEnergy(np.real(audio), 1, 60, 180, fs)
    print(bpm)
    return bpm

# Putting everything together

In [17]:
def filterbankMethod (fs, audio):
    bandlimits = np.array([0, 200, 400, 800, 1600, 3200])
    signal = filterbank(audio, bandlimits, fs)
    audio = hwindow(signal, 1, bandlimits, fs)
    audio = diffrect(audio)
    bpm = combfilter(audio, 1, 60, 180, len(bandlimits), fs)
    print(bpm)
    return bpm

In [18]:
def energyMethod (fs, audio):
    beat = energy_beat(audio, fs)
    beat = diffrect(beat)
    bpm = combfilterEnergy(beat, 1, 60, 180, fs)
    print(bpm)
    return bpm

In [19]:
def DWTMethod (fs, audio):
    audio = DWT(fs, audio) # cascaded Discrete Wavelet Transform, 6 bands
    #audio = LPF(audio) # lowpass filter
    audio = sig.hilbert(audio)
    audio = FWR(audio) # full wave rectification
    audio = downsample(audio)
    audio = normalize(audio)
    audio = autocorrelate(fs, audio)
    bpm = combfilterEnergy(audio, 2, 120, 360, fs)
    return bpm

## GUI Implementation

In [48]:
## This cell implements the GUI for you to enjoy our demo. Continue scrolling to run the demo :-)
im1 = imread('Tree1.png')
im2 = imread('Tree2.png')

from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider, Select, VBox,IntRangeSlider

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

method = Dropdown(options=['Energy','Filterbank','Spectrum'])
song = Dropdown(options=['bottle_60bpm.wav',
                         'Music_database/Rise.wav',
                         'Music_database/Despacito (feat. Daddy Yankee).wav',
                         'Music_database/In the Air Tonight.wav',
                         'Music_database/Opposite of Adults.wav',
                         'Music_database/Reality (feat. Janieck Devy).wav',
                         'Music_database/Rolling In the Deep.wav',
                         'Music_database/Sofia.wav',
                         'Music_database/Summer Paradise (feat. Sean Paul).wav',
                         'Music_database/Sunshine Road.wav',
                         'Music_database/Unity.wav',
                         'Music_database/Some Nights.wav',
                         'Music_database/American.wav',
                         'Music_database/Gold On the Ceiling.wav',
                         'Music_database/Ain\'t No Love In the Heart of the City.wav',
                         'Music_database/Lazy Bones.wav',
                         'Music_database/Santa Claus Is Coming To Town.wav',
                         'Music_database/Hide and Seek.wav',
                         'Music_database/I\'LL BE GONE.wav',
                         'Music_database/X-Kid.wav',
                         'Music_database/Nero\'s Nocturne.wav',
                         'Music_database/Highway to Hell.wav',
                         'Music_database/Remember to Breathe.wav',
                         'Music_database/Someone Like You.wav',
                         'Music_database/Hollywood.wav',
                         'Music_database/Jingle Bells (lyrics).wav',
                         'Music_database/All I Want For Christmas Is You.wav',
                         'Music_database/Christmas (Baby Please Come Home).wav',
                        ])
run_button = Button(description='Run', disabled=False,
                                        button_style='',
                                        tooltip='Click me',
                                        icon='check')

seconds = IntRangeSlider(
    value=[0, 8],
    min=0,
    max=240,
    step=1,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

form_items = [
    Box([Label(value='Method'), method], layout=form_item_layout),
    Box([Label(value='Song'), song], layout=form_item_layout),
    Box([Label(value='Seconds to analyse'), seconds], layout=form_item_layout),
    Box([run_button], layout=form_item_layout),

]

out = widgets.Output()

outb = [
    Box([out],layout=form_item_layout)
]

outbox = Box(outb, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='50%'
))

out2 = widgets.Output()

outb2 = [
    Box([out2],layout=form_item_layout)
]

outbox2 = Box(outb2, layout=Layout(
    display='',
    flex_flow='column',
    border='',
    align_items='stretch',
    width='50%',
    height = '300px'
))

out3 = widgets.Output()

outb3 = [
    Box([out3],layout=form_item_layout)
]

outbox3 = Box(outb3, layout=Layout(
    display='flex',
    flex_flow='column',
    border='',
    align_items='stretch',
    width='50%',
))

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='50%'
))

def cristmas_trees(a):
    #try:
    #    led_ring_address = '/dev/cu.usbmodem144301'
    #    led_ring = neopixels.NeoPixels(usb_port=led_ring_address)
    #except IOError:
    #    raise IOError("Could not connect to LED ring!")
    
    b = True
    sleep_t = 60/a-0.051196840059710667
    while b == True:
        try:
            clear_output()
            plt.axis('off')
            imshow(im1)
            #led_ring.lightify_mono(rgb=[0, 150, 0])
            show()
            time.sleep(sleep_t)
            clear_output()
            plt.axis('off')
            imshow(im2)
            #led_ring.lightify_mono(rgb=[150, 150, 0])
            show()
            time.sleep(sleep_t)
            
        except KeyboardInterrupt:
            b = False
            print('Manual break by user')
            pygame.mixer.music.stop() 
                       

def run(a):
    out.clear_output()
    with out:
        print('The detected BPM is: ')
        [fs, audio] = importAudio(song.value)
        if(seconds.value[1] < np.floor(len(audio)/fs)):
            audio = clipAudio(fs, audio, seconds.value)
        if(method.value=='Energy'):
            a = energyMethod(fs,audio)
        if(method.value=='Filterbank'):
            a = filterbankMethod(fs, audio)
        if(method.value=='Spectrum'):
            a = spectrumMethod(fs, audio)
    with out3:
        pygame.mixer.init()
        pygame.mixer.music.load(song.value)
        pygame.mixer.music.play()  
    with out2:
        cristmas_trees(a)


Beat_detection = VBox([form, outbox, outbox3, outbox2])
            
run_button.on_click(run)
        
   

## Running the Program with GUI
Finally, it's time to checkout the actual program!
Here's how you can test the program. 
1. Run the cell below.
2. Choose a beat detecting method from the dropdown menu
3. Choose a song from the dropdown menu
4. Drag the slider to specify which section of the song to analyze. As you'll see in our test, 8 second clip will suffice for most songs
5. Enjoy the Christmas tree! Audio will start playing as soon as bpm analysis is done. To stop the music, you need to run the next cell.

In [49]:
Beat_detection

A Jupyter Widget

In [41]:
pygame.mixer.music.stop()

Because imshow() function takes time that we can't account for, we've fine tuned the loop's pause time to make the visuals line up with the beat. It may not work on your computer so we uploaded a video for your entertainment. Check it out in the attached files :)

# References
1. Tzanetakis, George, and Perry Cook. "Musical genre classification of audio signals." IEEE Transactions on speech and audio processing 10.5 (2002): 293-302.
2. Scheirer, Eric D. "Tempo and beat analysis of acoustic musical signals." The Journal of the Acoustical Society of America 103.1 (1998): 588-601.
3. http://archive.gamedev.net/archive/reference/programming/features/beatdetection/index.html
4. 

